In [1]:
import pandas as pd

In [2]:
df_original.head()

NameError: name 'df_original' is not defined

In [3]:
asos = pd.read_csv('asos/power.csv', encoding='cp949')
asos.head()

,날짜,발전기명,용량_메가와트,01시,02시,03시,04시,05시,06시,07시,...,18시,19시,20시,21시,22시,23시,24시,합계,주소지,asos_num
0,2022-12-31,영암에프원태양광b,13.296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,153600.0,0.0,0.0,0.0,0.0,0.0,0.0,55008000,전라남도 영암군 삼호읍 에프원로 2 영암 F1 태양광 발전소,165
1,2022-12-30,영암에프원태양광b,13.296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,124800.0,0.0,0.0,0.0,0.0,0.0,0.0,47587200,전라남도 영암군 삼호읍 에프원로 2 영암 F1 태양광 발전소,165
2,2022-12-29,영암에프원태양광b,13.296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,105600.0,0.0,0.0,0.0,0.0,0.0,0.0,49497600,전라남도 영암군 삼호읍 에프원로 2 영암 F1 태양광 발전소,165
3,2022-12-28,영암에프원태양광b,13.296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,134400.0,0.0,0.0,0.0,0.0,0.0,0.0,37660800,전라남도 영암군 삼호읍 에프원로 2 영암 F1 태양광 발전소,165
4,2022-12-27,영암에프원태양광b,13.296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,115200.0,0.0,0.0,0.0,0.0,0.0,0.0,53270400,전라남도 영암군 삼호읍 에프원로 2 영암 F1 태양광 발전소,165


In [4]:
#df_original = pd.read_csv('power/한국서부발전(주)_신재생에너지발전량_20220228.csv', encoding='cp949')
# Adjusting for the 24시 issue during the combination of date and time
hours = [f'{i:02d}시' for i in range(1, 25)]
adjusted_dates = []
adjusted_powers = []
adjusted_years = []
adjusted_addresses = []
adjusted_capacities = []
adjusted_name = []
asos_num = []

for index, row in asos.iterrows():
    for hour in hours:
        if hour == "24시":
            adjusted_date = pd.Timestamp(row['날짜']) + pd.Timedelta(days=1)
            adjusted_hour = "00시"
        else:
            adjusted_date = pd.Timestamp(row['날짜'])
            adjusted_hour = hour
        
        power = row[hour]
        adjusted_dates.append(adjusted_date + pd.Timedelta(hours=int(adjusted_hour.replace('시', ''))))
        adjusted_powers.append(power)
        adjusted_years.append(adjusted_date.year)
        adjusted_name.append(row['발전기명'])
        adjusted_addresses.append(row['주소지'])
        adjusted_capacities.append(row['용량_메가와트'])
        asos_num.append(row['asos_num'])

# Constructing the final dataframe with additional columns
asos_new = pd.DataFrame({
    'year': adjusted_years,
    'power': adjusted_powers,
    'date': adjusted_dates,
    'name' : adjusted_name,
    'address': adjusted_addresses,
    'capacities': adjusted_capacities,
    'asos_num' : asos_num
})

asos_new = asos_new.drop(columns = 'year')
asos_new = asos_new[asos_new['power'] != 0.0]
asos_new['land'] = asos_new['capacities']*13200
asos_new['power/land'] = asos_new['power']/asos_new['land']
asos_new = asos_new.drop(columns = 'address')
asos_new.to_csv("asos_new.csv", index = False, encoding = 'cp949')

In [5]:
asos_new = pd.read_csv('asos_new.csv', encoding = 'cp949')
asos_new.head()

,power,date,name,capacities,asos_num,land,power/land
0,1296000.0,2022-12-31 09:00:00,영암에프원태양광b,13.296,165,175507.2,7.384312
1,4377600.0,2022-12-31 10:00:00,영암에프원태양광b,13.296,165,175507.2,24.942566
2,7065600.0,2022-12-31 11:00:00,영암에프원태양광b,13.296,165,175507.2,40.258177
3,8860800.0,2022-12-31 12:00:00,영암에프원태양광b,13.296,165,175507.2,50.486818
4,7891200.0,2022-12-31 13:00:00,영암에프원태양광b,13.296,165,175507.2,44.962258


In [6]:
asos_new['asos_num'].unique()

array([165, 112, 140, 133, 288, 239, 119, 260, 232], dtype=int64)

In [7]:
asos_new['name'].unique()

array(['영암에프원태양광b', '[서인천]발전부지내 태양광 1단계', '[서인천]발전부지내 태양광 1단계ESS',
       '[서인천]발전부지내 태양광 2단계', '군산복합2단계태양광', '군산복합2단계태양광ESS', '대전 학하 연료전지',
       '삼랑진태양광#1', '삼랑진태양광#2', '서인천연료전지', '서인천연료전지2',
       '세종시수질복원센터태양광연계 ESS', '수질복원센터태양광', '의왕 연료전지', '장흥풍력', '천안청수연료전지'],
      dtype=object)

In [93]:
import os
import zipfile
import pandas as pd

# 압축 푸는 함수
def unzip_files(directory):
    for item in os.listdir(directory): # 폴더 내의 모든 파일/디렉토리에 대해
        item_path = os.path.join(directory, item)
        if zipfile.is_zipfile(item_path): # 압축 파일인 경우
            with zipfile.ZipFile(item_path, 'r') as zip_ref:
                zip_ref.extractall(directory)

# 모든 CSV 파일을 하나의 데이터프레임으로 합치는 함수
def combine_csvs(directory):
    dfs = [] # 각 CSV 파일의 데이터프레임을 저장할 리스트
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                dfs.append(pd.read_csv(file_path, encoding='cp949'))  # encoding 옵션 수정

    return pd.concat(dfs, ignore_index=True)

# 주요 실행 코드
directory = './asos/asos' # 압축 파일들이 있는 디렉토리 경로를 입력하세요.

# 압축 풀기
unzip_files(directory)

# CSV 파일들 합치기
combined_df = combine_csvs(directory)
combined_df.to_csv(os.path.join(directory, 'asos_combined.csv'), index=False, encoding = 'cp949') # 합친 결과를 CSV로 저장

In [94]:
asos_combined.head()

,지점,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),...,운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,112,2017-01-01 00:00:00,3.1,NaN,2.2,200.0,81.0,6.2,0.1,1020.5,...,NaN,NaN,328.0,NaN,19.0,-0.2,1.1,1.3,2.2,2.8
1,112,2017-01-01 01:00:00,2.9,NaN,2.5,180.0,83.0,6.2,0.2,1020.3,...,NaN,NaN,318.0,NaN,19.0,-0.2,1.1,1.3,2.3,2.9
2,112,2017-01-01 02:00:00,2.3,NaN,2.2,160.0,85.0,6.1,0.0,1020.2,...,NaN,NaN,277.0,NaN,19.0,-0.4,1.1,1.3,2.3,2.9
3,112,2017-01-01 03:00:00,1.9,NaN,2.1,180.0,86.0,6.0,-0.2,1020.4,...,NaN,NaN,270.0,NaN,19.0,-0.5,1.1,1.4,2.3,2.9
4,112,2017-01-01 04:00:00,1.6,NaN,2.2,160.0,87.0,6.0,-0.3,1020.2,...,NaN,NaN,307.0,NaN,19.0,-0.8,1.1,1.3,2.2,2.8


In [8]:
asos_new = pd.read_csv('asos_new.csv', encoding = 'cp949')
asos_combined = pd.read_csv('asos/asos/asos_combined.csv', encoding = 'cp949')
asos_new['date'] = pd.to_datetime(asos_new['date'])
asos_combined['일시'] = pd.to_datetime(asos_combined['일시'])
asos_new['asos_num'] = asos_new['asos_num'].astype(float)
asos = pd.merge(asos_new, asos_combined, 
                left_on=['date', 'asos_num'], 
                right_on=['일시', '지점'], 
                how='left')
asos.head()

C:\Users\sedag\AppData\Local\Temp\ipykernel_22160\2453003822.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  asos_combined = pd.read_csv('asos/asos/asos_combined.csv', encoding = 'cp949')


,power,date,name,capacities,asos_num,land,power/land,지점,일시,기온(°C),...,운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,1296000.0,2022-12-31 09:00:00,영암에프원태양광b,13.296,165.0,175507.2,7.384312,165.0,2022-12-31 09:00:00,0.1,...,NaN,NaN,1295.0,NaN,NaN,-0.5,2.5,3.3,4.2,4.7
1,4377600.0,2022-12-31 10:00:00,영암에프원태양광b,13.296,165.0,175507.2,24.942566,165.0,2022-12-31 10:00:00,1.7,...,NaN,NaN,1534.0,NaN,NaN,0.0,2.9,3.4,4.2,4.8
2,7065600.0,2022-12-31 11:00:00,영암에프원태양광b,13.296,165.0,175507.2,40.258177,165.0,2022-12-31 11:00:00,2.8,...,NaN,NaN,2617.0,NaN,NaN,3.8,3.7,3.7,4.2,4.8
3,8860800.0,2022-12-31 12:00:00,영암에프원태양광b,13.296,165.0,175507.2,50.486818,165.0,2022-12-31 12:00:00,4.1,...,NaN,NaN,2779.0,NaN,NaN,10.3,4.4,3.9,4.2,4.8
4,7891200.0,2022-12-31 13:00:00,영암에프원태양광b,13.296,165.0,175507.2,44.962258,165.0,2022-12-31 13:00:00,4.2,...,Sc,10.0,3442.0,NaN,NaN,6.1,4.7,4.3,4.3,4.7


In [9]:
asos_cleaned = asos.dropna(subset=['지점'])

In [10]:
asos_cleaned.tail(50)

,power,date,name,capacities,asos_num,land,power/land,지점,일시,기온(°C),...,운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
252079,568736.0,2018-11-24 07:00:00,서인천연료전지2,60.3,112.0,795960.0,0.714528,112.0,2018-11-24 07:00:00,1.0,...,StNs,8.0,160.0,NaN,19060501.0,0.1,5.0,5.6,6.7,7.5
252080,561792.0,2018-11-24 08:00:00,서인천연료전지2,60.3,112.0,795960.0,0.705804,112.0,2018-11-24 08:00:00,0.8,...,StNs,8.0,444.0,NaN,190605.0,0.0,4.9,5.5,6.7,7.5
252081,564480.0,2018-11-24 09:00:00,서인천연료전지2,60.3,112.0,795960.0,0.709181,112.0,2018-11-24 09:00:00,0.9,...,StNs,1.0,197.0,NaN,190605.0,0.0,4.8,5.4,6.7,7.5
252082,629888.0,2018-11-24 10:00:00,서인천연료전지2,60.3,112.0,795960.0,0.791356,112.0,2018-11-24 10:00:00,1.9,...,St,2.0,829.0,NaN,1905.0,0.0,4.7,5.4,6.7,7.5
252083,887488.0,2018-11-24 11:00:00,서인천연료전지2,60.3,112.0,795960.0,1.114991,112.0,2018-11-24 11:00:00,4.2,...,Sc,10.0,1219.0,NaN,19.0,5.4,4.6,5.3,6.6,7.4
252084,535024.0,2018-11-24 12:00:00,서인천연료전지2,60.3,112.0,795960.0,0.672174,112.0,2018-11-24 12:00:00,4.8,...,Sc,4.0,1104.0,NaN,NaN,6.6,4.7,5.3,6.6,7.4
252085,478576.0,2018-11-24 13:00:00,서인천연료전지2,60.3,112.0,795960.0,0.601256,112.0,2018-11-24 13:00:00,5.0,...,Sc,4.0,1007.0,NaN,NaN,6.6,5.0,5.4,6.5,7.3
252086,753536.0,2018-11-24 14:00:00,서인천연료전지2,60.3,112.0,795960.0,0.946701,112.0,2018-11-24 14:00:00,5.5,...,ScCi,5.0,868.0,NaN,19.0,8.0,5.3,5.5,6.5,7.3
252087,1400672.0,2018-11-24 15:00:00,서인천연료전지2,60.3,112.0,795960.0,1.759727,112.0,2018-11-24 15:00:00,6.5,...,Sc,14.0,1129.0,NaN,19.0,8.1,5.7,5.7,6.6,7.2
252088,1426432.0,2018-11-24 16:00:00,서인천연료전지2,60.3,112.0,795960.0,1.792090,112.0,2018-11-24 16:00:00,6.7,...,Ci,NaN,981.0,NaN,19.0,6.6,6.1,5.9,6.6,7.2


In [11]:
asos_cleaned['지점'].unique()

array([165., 112., 140., 133., 288., 239., 119., 260., 232.])

In [12]:
asos_cleaned.to_csv("asos.csv", index = False, encoding = 'cp949')

In [15]:
asos = pd.read_csv('asos.csv', encoding='cp949')

asos.rename(columns={
    '지점' : 'location',
    '기온(°C)': 'Temp(C)',
    '강수량(mm)': 'Prec(mm)',
    '풍속(m/s)':'Wind_speed(m/s)',
    '이슬점온도(°C)': 'Dew_Point(C)',
    '일조(hr)':'sunshine(hr)',
    '일사(MJ/m2)': 'Solar_Radiation(MJ/m2)',
    '적설(cm)': 'Snowfall(cm)',
    '전운량(10분위)': 'Cloud_Cover(1/10)',
    '풍향(16방위)': 'Wind_Direction(16 compass points)', 
    '일시': 'Date/Time', 
    '증기압(hPa)': 'Vapor_pressure(hPa)', 
    '현지기압(hPa)': 'Local Atmospheric Pressure(hPa)', 
    '해면기압(hPa)': 'Sea-level_Pressure(hPa)', 
    '운형(운형약어)': 'Cloud_Form', 
    '최저운고(100m )' : 'Lowest_Cloud_Height(100m)', 
    '지면상태(지면상태코드)':'Ground_Condition', 
    '현상번호(국내식)':'Phenomenon_Number', 
    '지면온도(°C)':'Ground Temp', 
    '5cm 지중온도(°C)':'5cm_Underground_Temp',
    '10cm 지중온도(°C)': '10cm_Underground_Temp', 
    '20cm 지중온도(°C)':'20cm_Underground_Temp', 
    '30cm 지중온도(°C)':'30cm_Underground_Temp', 
    '3시간신적설(cm)':'3-hour_Fresh_Snowfall', 
    '중하층운량(10분위)': 'Mid-Low_Cloud_Cover', 
    '시정(10m)' : 'Visibility(10m)', 
    '습도(%)': 'Humidity'
    
}, inplace=True)
asos.columns

Index(['power', 'date', 'name', 'capacities', 'asos_num', 'land', 'power/land',
       'location', 'Date/Time', 'Temp(C)', 'Prec(mm)', 'Wind_speed(m/s)',
       'Wind_Direction(16 compass points)', 'Humidity', 'Vapor_pressure(hPa)',
       'Dew_Point(C)', 'Local Atmospheric Pressure(hPa)',
       'Sea-level_Pressure(hPa)', 'sunshine(hr)', 'Solar_Radiation(MJ/m2)',
       'Snowfall(cm)', '3-hour_Fresh_Snowfall', 'Cloud_Cover(1/10)',
       'Mid-Low_Cloud_Cover', 'Cloud_Form', 'Lowest_Cloud_Height(100m)',
       'Visibility(10m)', 'Ground_Condition', 'Phenomenon_Number',
       'Ground Temp', '5cm_Underground_Temp', '10cm_Underground_Temp',
       '20cm_Underground_Temp', '30cm_Underground_Temp'],
      dtype='object')

In [16]:
len(asos)

247612

In [105]:
len(asos)

247612

In [17]:
asos.to_csv("asos_train.csv", index = False, encoding = 'cp949')

In [107]:
result = asos.groupby('location')['date'].agg(['min', 'max'])
print(result)

                          min                  max
location                                          
112.0     2018-01-01 09:00:00  2022-03-01 00:00:00
119.0     2021-06-12 00:00:00  2022-03-01 00:00:00
133.0     2021-11-11 14:00:00  2022-03-01 00:00:00
140.0     2018-01-01 09:00:00  2022-03-01 00:00:00
165.0     2013-03-19 07:00:00  2022-12-31 18:00:00
232.0     2019-11-19 23:00:00  2022-03-01 00:00:00
239.0     2020-01-01 00:00:00  2022-03-01 00:00:00
260.0     2021-07-13 09:00:00  2022-03-01 00:00:00
288.0     2018-01-01 08:00:00  2022-02-28 19:00:00


In [18]:
asos = pd.read_csv('asos_train.csv', encoding='cp949')
asos.head()

,power,date,name,capacities,asos_num,land,power/land,location,Date/Time,Temp(C),...,Cloud_Form,Lowest_Cloud_Height(100m),Visibility(10m),Ground_Condition,Phenomenon_Number,Ground Temp,5cm_Underground_Temp,10cm_Underground_Temp,20cm_Underground_Temp,30cm_Underground_Temp
0,1296000.0,2022-12-31 09:00:00,영암에프원태양광b,13.296,165.0,175507.2,7.384312,165.0,2022-12-31 09:00:00,0.1,...,NaN,NaN,1295.0,NaN,NaN,-0.5,2.5,3.3,4.2,4.7
1,4377600.0,2022-12-31 10:00:00,영암에프원태양광b,13.296,165.0,175507.2,24.942566,165.0,2022-12-31 10:00:00,1.7,...,NaN,NaN,1534.0,NaN,NaN,0.0,2.9,3.4,4.2,4.8
2,7065600.0,2022-12-31 11:00:00,영암에프원태양광b,13.296,165.0,175507.2,40.258177,165.0,2022-12-31 11:00:00,2.8,...,NaN,NaN,2617.0,NaN,NaN,3.8,3.7,3.7,4.2,4.8
3,8860800.0,2022-12-31 12:00:00,영암에프원태양광b,13.296,165.0,175507.2,50.486818,165.0,2022-12-31 12:00:00,4.1,...,NaN,NaN,2779.0,NaN,NaN,10.3,4.4,3.9,4.2,4.8
4,7891200.0,2022-12-31 13:00:00,영암에프원태양광b,13.296,165.0,175507.2,44.962258,165.0,2022-12-31 13:00:00,4.2,...,Sc,10.0,3442.0,NaN,NaN,6.1,4.7,4.3,4.3,4.7


In [112]:
# 'location' 값이 232.0인 항목 추출
asos_232 = asos[asos['location'] == 232.0]

# 추출한 항목을 다른 CSV 파일로 저장
asos_232.to_csv('test_chenan.csv', index=False, encoding='cp949')

# 원본 데이터에서 'location' 값이 232.0인 항목 제거
asos_train_without_chenan = asos[asos['location'] != 232.0]

# 수정된 원본 데이터 저장
asos_train_without_chenan.to_csv('train_without_chenan.csv', index=False, encoding='cp949')